In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

url = "https://berlin.r.mikatiming.com/2024/?page=1&event=BML_HCH3C0OH266&event_main_group=BMW+BERLIN+MARATHON&pid=search&search%5Bage_class%5D=%25&search%5Bsex%5D=%25&search%5Bnation%5D=%25&search_sort=place_nosex"
driver.get(url)
time.sleep(4)

resultados = []

# Solo primeros 10 atletas
atletas = driver.find_elements(By.CSS_SELECTOR, "li.list-group-item.row")[:10]

for atleta in atletas:
    try:
        place = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-place.place-secondary").text
    except:
        place = "Sin place"

    try:
        piac = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-custom").text
    except:
        piac = "Sin PI.AC"

    try:
        name_pais = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-fullname a").text
        nombre = re.sub(r"\s*\([A-Z]{3}\)", "", name_pais).strip()
        pais_match = re.search(r"\(([A-Z]{3})\)", name_pais)
        pais = pais_match.group(1) if pais_match else "Sin país"
    except:
        nombre = "Sin nombre"
        pais = "Sin país"

    try:
        age_group = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-age_class").text.strip()
        age_group = age_group.replace("Age Group", "").strip()
    except:
        age_group = "Sin grupo"

    try:
        finish_block = atleta.find_element(By.XPATH, ".//div[contains(@class, 'type-time') and .//div[text()='Finish']]")
        html = finish_block.get_attribute("innerHTML")
        match = re.search(r'</div>\s*([0-9:]+)', html)
        finish = match.group(1).strip() if match else "Sin tiempo"
    except:
        finish = "Sin tiempo"

    resultados.append({
        "place": place,
        "pi.ac": piac,
        "nombre": nombre,
        "pais": pais,
        "age_group": age_group,
        "finish": finish
    })

driver.quit()

df = pd.DataFrame(resultados)

In [2]:
df

,place,pi.ac,nombre,pais,age_group,finish
0,Place,,Sin nombre,Sin país,,Sin tiempo
1,1,,"Mengesha, Milkesa",ETH,H,02:03:17
2,2,,"Kotut, Cybrian",KEN,30,02:03:22
3,3,,"Alew, Haymanot",ETH,H,02:03:31
4,4,,"Kiprop, Stephen",KEN,H,02:03:37
5,5,,"Kiros, Hailemariyam",ETH,H,02:04:35
6,6,,"Ikeda, Yohei",JPN,H,02:05:12
7,7,,"Takele, Tadese",ETH,H,02:05:13
8,8,,"Ruesom, Oqbe Kibrom",ERI,H,02:05:37
9,9,,"Onchari, Enock",KEN,H,02:05:53


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re

def scrape_berlin_test(genero, url):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)

    print(f"\n🔍 Scrapeando 10 atletas – Berlín – Género: {genero}")
    driver.get(url)
    time.sleep(4)

    resultados = []
    atletas = driver.find_elements(By.CSS_SELECTOR, "li.list-group-item.row")[:10]

    for atleta in atletas:
        try:
            place = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-place.place-secondary").text
        except:
            place = "Sin lugar"

        try:
            piac = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-custom").text
        except:
            piac = "Sin PI.AC"

        try:
            name_pais = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-fullname a").text
            nombre = re.sub(r"\s*\([A-Z]{3}\)", "", name_pais).strip()
            pais_match = re.search(r"\(([A-Z]{3})\)", name_pais)
            pais = pais_match.group(1) if pais_match else "Sin país"
        except:
            nombre = "Sin nombre"
            pais = "Sin país"

        try:
            age_group = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-age_class").text
            age_group = age_group.replace("Age Group", "").strip()
        except:
            age_group = "Sin grupo"

        try:
            finish_block = atleta.find_element(By.XPATH, ".//div[contains(@class, 'type-time') and .//div[text()='Finish']]")
            html = finish_block.get_attribute("innerHTML")
            match = re.search(r'</div>\s*([0-9:]+)', html)
            finish = match.group(1).strip() if match else "Sin tiempo"
        except:
            finish = "Sin tiempo"

        resultados.append({
            "place": place,
            "pi.ac": piac,
            "nombre": nombre,
            "pais": pais,
            "age_group": age_group,
            "finish": finish,
            "gender": genero
        })

    driver.quit()
    return pd.DataFrame(resultados)

# URLs
url_hombres = "https://berlin.r.mikatiming.com/2024/?page=1&event=BML_HCH3C0OH266&event_main_group=BMW+BERLIN+MARATHON&num_results=100&pid=list&search%5Bsex%5D=M&search%5Bage_class%5D=%25"
url_mujeres = "https://berlin.r.mikatiming.com/2024/?page=1&event=BML_HCH3C0OH266&event_main_group=BMW+BERLIN+MARATHON&num_results=100&pid=list&search%5Bsex%5D=W&search%5Bage_class%5D=%25"

# Ejecutar prueba
df_hombres = scrape_berlin_test("M", url_hombres)
df_mujeres = scrape_berlin_test("W", url_mujeres)

# Concatenar
df_berlin = pd.concat([df_hombres, df_mujeres], ignore_index=True)

print("\n✅ Prueba Berlín completada. Archivo: 'prueba_berlin_10_M_10_W.csv'")


🔍 Scrapeando 10 atletas – Berlín – Género: M

🔍 Scrapeando 10 atletas – Berlín – Género: W

✅ Prueba Berlín completada. Archivo: 'prueba_berlin_10_M_10_W.csv'


In [5]:
df_berlin

,place,pi.ac,nombre,pais,age_group,finish,gender
0,Pl.AC,,Sin nombre,Sin país,,Sin tiempo,M
1,1,,"Mengesha, Milkesa",ETH,H,02:03:17,M
2,1,,"Kotut, Cybrian",KEN,30,02:03:22,M
3,2,,"Alew, Haymanot",ETH,H,02:03:31,M
4,3,,"Kiprop, Stephen",KEN,H,02:03:37,M
5,4,,"Kiros, Hailemariyam",ETH,H,02:04:35,M
6,5,,"Ikeda, Yohei",JPN,H,02:05:12,M
7,6,,"Takele, Tadese",ETH,H,02:05:13,M
8,7,,"Ruesom, Oqbe Kibrom",ERI,H,02:05:37,M
9,8,,"Onchari, Enock",KEN,H,02:05:53,M


In [6]:
df_berlin.to_csv("../data/ejemplo_10_resultados_berlinmarathon_2024.csv", index=False)

### SCRAPEO FINAL

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re
from tqdm import tqdm

def scrape_berlin_completo(genero, url_base, paginas, archivo_salida):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)

    resultados = []

    for pagina in tqdm(range(1, paginas + 1), desc=f"Berlín {genero}", ncols=100):
        url = f"{url_base}&page={pagina}"
        driver.get(url)
        time.sleep(3.5)

        atletas = driver.find_elements(By.CSS_SELECTOR, "li.list-group-item.row")

        for atleta in atletas:
            try:
                place = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-place.place-secondary").text
            except:
                place = "Sin lugar"

            try:
                piac = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-custom").text
            except:
                piac = "Sin PI.AC"

            try:
                name_pais = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-fullname a").text
                nombre = re.sub(r"\s*\([A-Z]{3}\)", "", name_pais).strip()
                pais_match = re.search(r"\(([A-Z]{3})\)", name_pais)
                pais = pais_match.group(1) if pais_match else "Sin país"
            except:
                nombre = "Sin nombre"
                pais = "Sin país"

            try:
                age_group = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-age_class").text
                age_group = age_group.replace("Age Group", "").strip()
            except:
                age_group = "Sin grupo"

            try:
                finish_block = atleta.find_element(By.XPATH, ".//div[contains(@class, 'type-time') and .//div[text()='Finish']]")
                html = finish_block.get_attribute("innerHTML")
                match = re.search(r'</div>\s*([0-9:]+)', html)
                finish = match.group(1).strip() if match else "Sin tiempo"
            except:
                finish = "Sin tiempo"

            resultados.append({
                "page": pagina,
                "place": place,
                "pi.ac": piac,
                "nombre": nombre,
                "pais": pais,
                "age_group": age_group,
                "finish": finish,
                "gender": genero
            })

        # Autoguardado cada 5 páginas
        if pagina % 5 == 0 or pagina == paginas:
            df_parcial = pd.DataFrame(resultados)
            df_parcial.to_csv(archivo_salida, index=False)

    driver.quit()
    return pd.DataFrame(resultados)

# URLs base
url_hombres = "https://berlin.r.mikatiming.com/2024/?event=BML_HCH3C0OH266&event_main_group=BMW+BERLIN+MARATHON&num_results=100&pid=list&search%5Bsex%5D=M&search%5Bage_class%5D=%25"
url_mujeres = "https://berlin.r.mikatiming.com/2024/?event=BML_HCH3C0OH266&event_main_group=BMW+BERLIN+MARATHON&num_results=100&pid=list&search%5Bsex%5D=W&search%5Bage_class%5D=%25"

# Ejecutar scraping por género
df_hombres = scrape_berlin_completo("M", url_hombres, paginas=356, archivo_salida="resultados_berlin_M_parcial.csv")
df_mujeres = scrape_berlin_completo("W", url_mujeres, paginas=186, archivo_salida="resultados_berlin_W_parcial.csv")

# Concatenar y guardar final
df_total_berlin = pd.concat([df_hombres, df_mujeres], ignore_index=True)
df_total_berlin.to_csv("../data/resultados_berlin_2024_completo.csv", index=False)

print("\n✅ ¡Scrapeo COMPLETO de Berlín finalizado! Archivo: 'resultados_berlin_2024_completo.csv'")

Berlín W: 100%|███████████████████████████████████████████████████| 186/186 [21:43<00:00,  7.01s/it]



✅ ¡Scrapeo COMPLETO de Berlín finalizado! Archivo: 'resultados_berlin_2024_completo.csv'


In [8]:
df_total_berlin

,page,place,pi.ac,nombre,pais,age_group,finish,gender
0,1,,,Sin nombre,Sin país,,Sin tiempo,M
1,1,,,"Mengesha, Milkesa",ETH,H,02:03:17,M
2,1,,,"Kotut, Cybrian",KEN,30,02:03:22,M
3,1,,,"Alew, Haymanot",ETH,H,02:03:31,M
4,1,,,"Kiprop, Stephen",KEN,H,02:03:37,M
...,...,...,...,...,...,...,...,...
54634,186,2234,,"Syamsiar, Laily",INA,50,07:46:53,W
54635,186,2800,,"Kaur, Sukhmanpreet",IND,H,07:48:40,W
54636,186,2798,,"Pompos, Taylor",USA,30,07:51:03,W
54637,186,2991,,"Belmezioud, Hakima",FRA,40,07:51:59,W
